# 导入库并且导入机器学习的数据

In [ ]:
import os

import tkinter as tk
from tkinter import filedialog
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt

import pandas._libs.testing as _testing


import tensorflow as tf
from tensorflow import keras


from tensorflow.keras import layers

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dense, Layer, Add
from tensorflow.keras.regularizers import l2




from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dense, Activation, GRU, LSTM, Dropout, Input, Embedding, RepeatVector, concatenate, Convolution2D, Conv2D, BatchNormalization, LayerNormalization, MultiHeadAttention, Conv1D, GlobalMaxPooling1D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from keras.utils import plot_model, CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  

from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score 
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import load_model


root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename()

folder_path = os.path.dirname(file_path)

file_name = os.path.basename(file_path)

df = pd.read_excel(file_path, sheet_name="MachineLearningData")


features = df.iloc[:, :-1].values   
target = df.iloc[:, -1].values  

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

scaler = StandardScaler()  #
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))




In [ ]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

In [ ]:


import time
from tensorflow.keras.callbacks import Callback
class TrainingTimeCallback(Callback):
    def on_train_begin(self, logs=None):
        self.start_time = time.time()
    def on_train_end(self, logs=None):
        total_time = time.time() - self.start_time
        print(f"Total training time: {total_time} seconds")


class ResidualBlock(Layer):

    def __init__(self, num_filters, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.num_filters = num_filters
        self.conv1 = Conv2D(num_filters, (3, 3), padding='same')
        self.bn1 = BatchNormalization()
        self.relu1 = Activation('relu')
        self.conv2 = Conv2D(num_filters, (3, 3), padding='same')
        self.bn2 = BatchNormalization()
        self.relu2 = Activation('relu')
        self.add = Add()

    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.add([x, input_tensor])
        return self.relu2(x)

    def get_config(self):
        config = super(ResidualBlock, self).get_config()
        config['num_filters'] = self.num_filters
        return config 
    

custom_objects = {
    'ResidualBlock': ResidualBlock
}

class TransformerEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerEncoderLayer, self).__init__()

        self.multi_head_attention = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.layer_norm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dense1 = tf.keras.layers.Dense(dff, activation='relu')
        self.dense2 = tf.keras.layers.Dense(d_model)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.layer_norm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs, training):
        attention_output = self.multi_head_attention(inputs, inputs)
        attention_output = self.dropout1(attention_output, training=training)
        attention_output = self.layer_norm1(inputs + attention_output)

        dense_output = self.dense1(attention_output)
        dense_output = self.dense2(dense_output)
        dense_output = self.dropout2(dense_output, training=training)
        output = self.layer_norm2(attention_output + dense_output)
        return output
    

class AttentionLayer(Layer):
    def __init__(self, num_heads, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)
        self.num_heads = num_heads

    def build(self, input_shape):

        batch_size, seq_len, embedding_dim = input_shape
   
        self.head_dim = embedding_dim // self.num_heads

        tf.debugging.assert_equal(embedding_dim % self.num_heads, 0,
                                  message='Embedding dimension must be divisible by the number of heads.')

        self.query_dense = Dense(embedding_dim)
        self.key_dense = Dense(embedding_dim)
        self.value_dense = Dense(embedding_dim)

        self.multihead_attention = MultiHeadAttention(self.num_heads, self.head_dim)
        self.dense = Dense(embedding_dim)

    def call(self, inputs):

        queries = self.query_dense(inputs)
        keys = self.key_dense(inputs)
        values = self.value_dense(inputs)

        attention_output = self.multihead_attention(queries, keys, values)

        output = self.dense(attention_output)
        return output
    
class PositionalEncoding(layers.Layer):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        d_model = tf.shape(inputs)[2]
        position = tf.math.cumsum(tf.ones_like(inputs[:, :, 0]), axis=1, exclusive=True)
        position_enc = tf.cast(position, tf.float32)[:, tf.newaxis, :]
        div_term = tf.exp(tf.range(0, tf.cast(d_model, tf.float32), 2) * -(tf.math.log(10000.0) / tf.cast(d_model, tf.float32)))
        div_term = div_term[tf.newaxis, tf.newaxis, :]
        position_enc = position_enc * div_term
        position_enc = tf.concat([tf.math.sin(position_enc), tf.math.cos(position_enc)], axis=-1)
        position_enc = tf.expand_dims(position_enc, axis=2)
        position_enc = tf.reshape(position_enc, tf.shape(inputs))
        return inputs + position_enc
    
def transformer_block(inputs, num_heads, dff, dropout_rate):

    attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(inputs, inputs)
    attention = layers.Dropout(dropout_rate)(attention)
    attention = layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    outputs = layers.Dense(units=dff, activation="relu")(attention)
    outputs = layers.Dense(units=d_model)(outputs)
    outputs = layers.Dropout(dropout_rate)(outputs)
    outputs = layers.LayerNormalization(epsilon=1e-6)(attention + outputs)
    return outputs

In [ ]:
model_select = 'CNN'  

# All models

In [ ]:



elif model_select == 'DNN':
    ## DNN
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    history = model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[TrainingTimeCallback()])
    model.save(f'{model_select}-model.h5')

elif model_select == 'LSTM':
    # LSTM
    model = Sequential()
    model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[TrainingTimeCallback()])
    model.save(f'{model_select}-model.h5')
    
elif model_select == 'GRU':
    # （GRU）
    model = Sequential()
    model.add(GRU(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[TrainingTimeCallback()])
    model.save(f'{model_select}-model.h5')
    
elif model_select == 'CNN':
    ##（CNN）
    model = Sequential()
    model.add(Conv1D(256, 3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(MaxPooling1D(1))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    history = model.fit(X_train, y_train, epochs=1000, batch_size=256, validation_data=(X_test, y_test), callbacks=[TrainingTimeCallback()])
    model.save(f'{model_select}-model.h5')





# training loss

In [ ]:

# val_loss
import matplotlib.pyplot as plt
import matplotlib

plt.rcParams['font.sans-serif'] = ['SimHei']  
plt.rcParams['axes.unicode_minus'] = False  
plt.plot(history.history['val_loss'])
plt.title(f'{model_select}-Model validation loss')
plt.ylim([1e-5, 10])  
plt.yscale('log')  
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.show()

  
data = []  
for i in range(len(history.history['val_loss'])):  
    data.append([i, history.history['val_loss'][i]])  
df = pd.DataFrame(data, columns=['Epoch', 'Loss'])  
  
    

output_folder = folder_path

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

file_name=os.path.splitext(file_name)[0]
file_name_pre = os.path.join(output_folder, f"{file_name}-{model_select}-Model Loss curve.xlsx")

if os.path.exists(file_name_pre):
    os.remove(file_name_pre)

df.to_excel(file_name_pre, index=False)

In [ ]:
y_pred = model.predict(X_test)

y_pred = np.squeeze(y_pred)


R2 = r2_score(y_test,y_pred) 
MAPE = mean_absolute_percentage_error(y_test,y_pred)
MSE = mean_squared_error(y_test,y_pred) 

data_metrics = { 
        'R2': R2,
        'MAPE': MAPE,
        'MSE': MSE,
    }
print (data_metrics)


plt.scatter(y_test, y_pred)
plt.plot([0, 8], [0, 8], color='black', linestyle='-', linewidth=1)
plt.title(f'{model_select}-Model Actual_strain vs Predicted_strain')
plt.xlabel('Actual_strain(%)')
plt.ylabel('Predicted_strain(%)')
plt.show()



df1 = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})  
df2 = pd.DataFrame(data_metrics, index=[0])  


file_name_pre = os.path.join(output_folder, f"{file_name}-{model_select}-Actual_strain vs Predicted_strain.xlsx")

if os.path.exists(file_name_pre):
    os.remove(file_name_pre)


df1.to_excel(file_name_pre, index=False)


file_name_pre = os.path.join(output_folder, f"{file_name}-{model_select}-R2-MAPE-MSE.xlsx")

if os.path.exists(file_name_pre):
    os.remove(file_name_pre)
df2.to_excel(file_name_pre, index=False)




## predict


In [ ]:

temperature_new=820 #℃
stress_new=350 #MPa
time_new =500 #h
point_number=500
X_new = np.c_[np.full(point_number, temperature_new), np.full(point_number, stress_new), np.linspace(0, time_new, point_number)]
X_new_scale = scaler.transform(X_new)

model = load_model(f'{model_select}-model.h5')

fig, ax = plt.subplots()
y_new_predict = model.predict(X_new_scale)

ax.plot(X_new[:, -1], y_new_predict, label= model_select, lw=0.5)#
ax.set_title(f'{temperature_new}℃/{stress_new}MPa  time-strain curve')
ax.set_ylim(0, 8)
plt.xlabel('Time(h)')
plt.ylabel('Strain(%)')
ax.legend()
plt.show()